<a href="https://colab.research.google.com/github/JLev-data/fcc_book_recommendation_knn/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [8]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-10-16 14:53:06--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M  21.9MB/s    in 1.1s    

2024-10-16 14:53:08 (21.9 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace BX-Users.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [9]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [10]:
# add your code here - consider creating a new cell for each section of code

verbose = 0

In [11]:
# Look at the data
for df in [df_books,df_ratings]:
  print(df.info())
  if verbose > 1:
    print(df.shape)
    print(df.describe())
  if verbose > 2:
    print(df)
  print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   isbn    271379 non-null  object
 1   title   271379 non-null  object
 2   author  271377 non-null  object
dtypes: object(3)
memory usage: 6.2+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   user    1149780 non-null  int32  
 1   isbn    1149780 non-null  object 
 2   rating  1149780 non-null  float32
dtypes: float32(1), int32(1), object(1)
memory usage: 17.5+ MB
None



To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

In [12]:
# remove non significant values
user_count = df_ratings.user.value_counts(sort=False)
valid_user = user_count[user_count >= 200].index

book_count  = df_ratings.isbn.value_counts(sort=False)
valid_book = book_count[book_count >= 100].index

df_ratings_clean = df_ratings[df_ratings.user.isin(valid_user) & df_ratings.isbn.isin(valid_book)].reset_index(drop=True)
df_books_clean = df_books[df_books.isbn.isin(valid_book)].reset_index(drop=True)

if verbose > 0:
  print(df_ratings_clean)

In [13]:
# merge dfs
df_merged = pd.merge(df_ratings_clean, df_books_clean[['isbn', 'title']], on='isbn', how='left')
if verbose > 0:
  print(df_merged)

In [14]:
# put isbn as index
book_ratings_matrix = df_merged.pivot_table(index='isbn', columns='user', values='rating').fillna(0)
if verbose > 0:
  print(book_ratings_matrix)

In [15]:
# Build and fit model
neigh = NearestNeighbors(n_neighbors=5,metric='cosine')
neigh.fit(book_ratings_matrix)

NearestNeighbors(metric='cosine')

Notice that the data returned from get_recommends() is a list.
The first element in the list is the book title passed into the function.
The second element in the list is a list of five more lists.
Each of the five lists contains a recommended book and the distance from the recommended book to the book passed into the function.

In [16]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):

  recommended_books = [book, []]
  book_isbn = df_books_clean[df_books_clean['title'] == book]['isbn']

  dist, recommended_book = neigh.kneighbors(X=book_ratings_matrix.loc[book_isbn], n_neighbors=6, return_distance=True)
  for i in range(5,0,-1):
    recommended_isbn = book_ratings_matrix.iloc[recommended_book[0][i]].name
    recomm_book = df_books_clean[df_books_clean['isbn']==recommended_isbn]['title'].values[0]
    recommended_books[1].append([recomm_book , dist[0][i]])
  return recommended_books

In [17]:
#Test
if verbose > 0:
  for ele in get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")[1]:
    print(ele)
  print()
  for ele in get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")[1]:
    print(ele)

In [18]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
